In [1]:
import sys, os

# For .py
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
# For .ipynb
sys.path.append(os.path.dirname(os.getcwd()))

import json
import random
from util import Image_Loader
from util import find_random_question
from util import random_word
from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
import re
import ast

- 키워드 생성

In [2]:
default_model="gpt-4o-mini"
verbose=True

total_cost = 0
keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_2'
# random_question = find_random_question(keyword, problem_type, detail_type=1, count=2)
# random_question

### Create a model for generating a problem
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.5, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

### get random word
main_topic = random_word()
if verbose:
    print(main_topic)

## Generate a new word based on the random word
problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 명사 1개를 제시하라. no intro. no outro."),
        HumanMessage(content="주제어: " + main_topic + " 명사: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
main_topic_generated = word_responses[0][1]
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))


눈
gpt-4o-mini-2024-07-18 | 
안경

OpenAI Tokens Used: 48
    Prompt Tokens: 46
    Completion Tokens: 2
Successful Requests: 1
Total Cost (USD): $0.0001
Total Cost (KRW): ₩0.135
--------------------------------------------------------------------------------------------------


In [3]:
### Create a model for generating a problem
models = [default_model]
paragraph_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=1.0, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

- 논설문 생성

In [4]:
# answer_format = {'소설': 'TBD',
#                 '심정을 나타내는 대사': 'TBD'}
human_prompt = f"""주제어: {main_topic_generated}"""
display(human_prompt)
paragraph_generate_model.request_models_responses(
    [
        SystemMessage(
            content=f"너는 논설문을 쓰는 봇이다. 총 7개의 문장으로 구성하라."
        ),
        HumanMessage(content=human_prompt),
    ]
)

paragraph_response = paragraph_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', paragraph_response[0][2]).group(1))

paragraph_response = paragraph_response[0][1]
display(paragraph_response)
# target_phrase = None
# similar_phrase = None
# for phrase in phrase_list:
#     if phrase in sentence_response:
#         target_phrase = phrase
#     else:
#         similar_phrase = phrase
# assert target_phrase is not None, "Target phrase not found in the sentence"
# assert similar_phrase is not None, "Similar phrase not found in the sentence"
# print(f'Target Phrase: {target_phrase}\nSimilar Phrase: {similar_phrase}')

'주제어: 안경'

gpt-4o-mini-2024-07-18 | 
안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이다. 현대 사회에서 많은 사람들이 안경을 착용하며, 이는 단순한 시력 보정 이상의 의미를 가진다. 안경은 패션 아이템으로도 자리 잡아, 개인의 스타일을 표현하는 중요한 요소가 되었다. 또한, 다양한 종류의 렌즈와 프레임 디자인은 소비자에게 선택의 폭을 넓혀 준다. 그러나 안경의 사용은 주의가 필요하다, 잘못된 착용이나 관리로 인해 시력에 부정적인 영향을 미칠 수 있다. 앞으로 기술이 발전하면서 안경의 기능성은 더욱 향상될 것이며, 다양한 방법으로 우리의 삶의 질을 높일 것이다. 따라서 올바른 안경 착용과 관리는 누구에게나 필수적인 문제로 남아야 한다.

OpenAI Tokens Used: 224
    Prompt Tokens: 40
    Completion Tokens: 184
Successful Requests: 1
Total Cost (USD): $0.00045
Total Cost (KRW): ₩0.6075
--------------------------------------------------------------------------------------------------


'안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이다. 현대 사회에서 많은 사람들이 안경을 착용하며, 이는 단순한 시력 보정 이상의 의미를 가진다. 안경은 패션 아이템으로도 자리 잡아, 개인의 스타일을 표현하는 중요한 요소가 되었다. 또한, 다양한 종류의 렌즈와 프레임 디자인은 소비자에게 선택의 폭을 넓혀 준다. 그러나 안경의 사용은 주의가 필요하다, 잘못된 착용이나 관리로 인해 시력에 부정적인 영향을 미칠 수 있다. 앞으로 기술이 발전하면서 안경의 기능성은 더욱 향상될 것이며, 다양한 방법으로 우리의 삶의 질을 높일 것이다. 따라서 올바른 안경 착용과 관리는 누구에게나 필수적인 문제로 남아야 한다.'

- 필자 태도 생성

In [5]:
example_question = {
    '정답': ['과학 정책에 대한 정부의 지나친 개입을 경계하고 있다.'],
    '오답': ['과학 기술 발전을 위해서는 연구가 중요함을 강조하고 있다.', '과학 기술 발전이 경제 성장에 미치는 영향력에 감탄하고 있다.', '과학 정책 수립 시 우주 과학이 소홀히 다루어질 것을 우려하고 있다.']
    }
human_prompt = f"""논설문: {paragraph_response}"""
display(human_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content=f"주어진 논설문에서 필자의 태도를 알아내라. 필자의 태도를 나타내는 선택지 1개, 필자의 태도와 반대되는 선택지 3개를 만들어라. 예시: {example_question}"
        ),
        HumanMessage(content=human_prompt),
    ]
)

attitude_response = problem_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', attitude_response[0][2]).group(1))

attitude_response = attitude_response[0][1]
attitude_response = ast.literal_eval(attitude_response)
display(attitude_response)

'논설문: 안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이다. 현대 사회에서 많은 사람들이 안경을 착용하며, 이는 단순한 시력 보정 이상의 의미를 가진다. 안경은 패션 아이템으로도 자리 잡아, 개인의 스타일을 표현하는 중요한 요소가 되었다. 또한, 다양한 종류의 렌즈와 프레임 디자인은 소비자에게 선택의 폭을 넓혀 준다. 그러나 안경의 사용은 주의가 필요하다, 잘못된 착용이나 관리로 인해 시력에 부정적인 영향을 미칠 수 있다. 앞으로 기술이 발전하면서 안경의 기능성은 더욱 향상될 것이며, 다양한 방법으로 우리의 삶의 질을 높일 것이다. 따라서 올바른 안경 착용과 관리는 누구에게나 필수적인 문제로 남아야 한다.'

gpt-4o-mini-2024-07-18 | 
{'정답': ['안경의 올바른 착용과 관리의 중요성을 강조하고 있다.'], '오답': ['안경은 단순한 패션 아이템일 뿐이라고 주장하고 있다.', '안경의 기능성 향상에 대해 회의적인 시각을 가지고 있다.', '안경 착용이 시력에 미치는 부정적인 영향을 과장하고 있다.']}

OpenAI Tokens Used: 420
    Prompt Tokens: 335
    Completion Tokens: 85
Successful Requests: 1
Total Cost (USD): $0.00084
Total Cost (KRW): ₩1.134
--------------------------------------------------------------------------------------------------


{'정답': ['안경의 올바른 착용과 관리의 중요성을 강조하고 있다.'],
 '오답': ['안경은 단순한 패션 아이템일 뿐이라고 주장하고 있다.',
  '안경의 기능성 향상에 대해 회의적인 시각을 가지고 있다.',
  '안경 착용이 시력에 미치는 부정적인 영향을 과장하고 있다.']}

- 내용 일치 문제 생성

In [6]:
example_question = {
    '정답': ['많은 국가들이 신에너지 개발에 대한 투자를 줄이고 있다.'],
    '오답': ['과학 정책이 빠르게 변해서 과학 기술이 발전할 수 있었다.', '정부가 우주 산업에 대한 규제를 풀어 성장한 민간 기업이 있다.', '우주 개발에 참여 중인 민간 기업이 화성에 호텔을 건설하고 있다.']
    }
human_prompt = f"""
글: {paragraph_response}"""
display(human_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content=f"글의 내용을 요약한 문장 1개와 틀린 문장 3개를 만들어라. 예시: {example_question}"
        ),
        HumanMessage(content=human_prompt),
    ]
)

matching_content_response = problem_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', matching_content_response[0][2]).group(1))

matching_content_response = matching_content_response[0][1]
matching_content_response = ast.literal_eval(matching_content_response)
display(matching_content_response)

'\n글: 안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이다. 현대 사회에서 많은 사람들이 안경을 착용하며, 이는 단순한 시력 보정 이상의 의미를 가진다. 안경은 패션 아이템으로도 자리 잡아, 개인의 스타일을 표현하는 중요한 요소가 되었다. 또한, 다양한 종류의 렌즈와 프레임 디자인은 소비자에게 선택의 폭을 넓혀 준다. 그러나 안경의 사용은 주의가 필요하다, 잘못된 착용이나 관리로 인해 시력에 부정적인 영향을 미칠 수 있다. 앞으로 기술이 발전하면서 안경의 기능성은 더욱 향상될 것이며, 다양한 방법으로 우리의 삶의 질을 높일 것이다. 따라서 올바른 안경 착용과 관리는 누구에게나 필수적인 문제로 남아야 한다.'

gpt-4o-mini-2024-07-18 | 
{'정답': ['안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이며, 패션 아이템으로도 중요한 역할을 한다.'], '오답': ['안경은 오직 시력 보정만을 위한 도구로 사용된다.', '모든 사람들이 안경을 착용하는 것은 아니다.', '안경의 사용은 시력에 긍정적인 영향을 미친다.']}

OpenAI Tokens Used: 403
    Prompt Tokens: 312
    Completion Tokens: 91
Successful Requests: 1
Total Cost (USD): $0.00081
Total Cost (KRW): ₩1.0935
--------------------------------------------------------------------------------------------------


{'정답': ['안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이며, 패션 아이템으로도 중요한 역할을 한다.'],
 '오답': ['안경은 오직 시력 보정만을 위한 도구로 사용된다.',
  '모든 사람들이 안경을 착용하는 것은 아니다.',
  '안경의 사용은 시력에 긍정적인 영향을 미친다.']}

In [7]:
result1 = {
    "example": paragraph_response,
    "selector": attitude_response['정답'] + attitude_response['오답'],
    "answer": attitude_response['정답'],
    "eval_answer": attitude_response['정답'],
    "total_cost": total_cost,
}
result1

{'example': '안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이다. 현대 사회에서 많은 사람들이 안경을 착용하며, 이는 단순한 시력 보정 이상의 의미를 가진다. 안경은 패션 아이템으로도 자리 잡아, 개인의 스타일을 표현하는 중요한 요소가 되었다. 또한, 다양한 종류의 렌즈와 프레임 디자인은 소비자에게 선택의 폭을 넓혀 준다. 그러나 안경의 사용은 주의가 필요하다, 잘못된 착용이나 관리로 인해 시력에 부정적인 영향을 미칠 수 있다. 앞으로 기술이 발전하면서 안경의 기능성은 더욱 향상될 것이며, 다양한 방법으로 우리의 삶의 질을 높일 것이다. 따라서 올바른 안경 착용과 관리는 누구에게나 필수적인 문제로 남아야 한다.',
 'selector': ['안경의 올바른 착용과 관리의 중요성을 강조하고 있다.',
  '안경은 단순한 패션 아이템일 뿐이라고 주장하고 있다.',
  '안경의 기능성 향상에 대해 회의적인 시각을 가지고 있다.',
  '안경 착용이 시력에 미치는 부정적인 영향을 과장하고 있다.'],
 'answer': ['안경의 올바른 착용과 관리의 중요성을 강조하고 있다.'],
 'eval_answer': ['안경의 올바른 착용과 관리의 중요성을 강조하고 있다.'],
 'total_cost': 2.9699999999999998}

In [8]:
result2 = {
    "example": paragraph_response,
    "selector": matching_content_response['정답'] + matching_content_response['오답'],
    "answer": matching_content_response['정답'],
    "eval_answer": matching_content_response['정답'],
    "total_cost": total_cost,
}
result2

{'example': '안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이다. 현대 사회에서 많은 사람들이 안경을 착용하며, 이는 단순한 시력 보정 이상의 의미를 가진다. 안경은 패션 아이템으로도 자리 잡아, 개인의 스타일을 표현하는 중요한 요소가 되었다. 또한, 다양한 종류의 렌즈와 프레임 디자인은 소비자에게 선택의 폭을 넓혀 준다. 그러나 안경의 사용은 주의가 필요하다, 잘못된 착용이나 관리로 인해 시력에 부정적인 영향을 미칠 수 있다. 앞으로 기술이 발전하면서 안경의 기능성은 더욱 향상될 것이며, 다양한 방법으로 우리의 삶의 질을 높일 것이다. 따라서 올바른 안경 착용과 관리는 누구에게나 필수적인 문제로 남아야 한다.',
 'selector': ['안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이며, 패션 아이템으로도 중요한 역할을 한다.',
  '안경은 오직 시력 보정만을 위한 도구로 사용된다.',
  '모든 사람들이 안경을 착용하는 것은 아니다.',
  '안경의 사용은 시력에 긍정적인 영향을 미친다.'],
 'answer': ['안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이며, 패션 아이템으로도 중요한 역할을 한다.'],
 'eval_answer': ['안경은 시력을 보호하고 향상시키기 위한 필수적인 도구이며, 패션 아이템으로도 중요한 역할을 한다.'],
 'total_cost': 2.9699999999999998}